In [248]:
from dotenv import load_dotenv
import os
load_dotenv()
spotify_client_id = os.getenv("SPOTIFY_CLIENT_ID")
spotify_client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(spotify_client_id, spotify_client_secret))
import pandas as pd
import numpy as np
import re
import ast
def search_artist(string = "--"):
    results = spotify.search(q='artist:'+string, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        artist = items[0]
        df_artist = pd.Series(artist)
        df_artist = df_artist[df_artist.index.isin(['name', 'followers', 'genres', 'id', 'popularity'])]
        df_artist["followers"] = df_artist["followers"]['total']
        return df_artist
def col(df, colname = "artist_name"):
    return np.array([int(x == colname) for x in df.columns]).argmax()
def query_track(df, string = "--", full = True):
    for i, track in enumerate(df["track_name"]):
        if(len(re.findall("(.*?)"+string+"(.*?)", "".join(track))) != 0):
            if(full):
                print(df.iloc[i], "\n")
            else:
                print("Name: ", df.iloc[i, col(df, "track_name")], "\nId: ", df.iloc[i, col(df, "track_id")],
                    "\nArtist: ", df.iloc[i, col(df)])
def normalize(df):    
    df["popularity"] = df["popularity"]/100
    df["tempo"] = round(df["tempo"]/20).astype(int)
    df["loudness"] = round((df["loudness"]+60)/10).astype(int)
    df["loudness"][df["loudness"]<0] == 0
    df["duration_ms"] = round(df["duration_ms"]/60000).astype(int)
    df = df.rename(columns = {"duration_ms": "duration_min"})
    return df
def show_corr(df, thr = 0.5):
    return ((pd.DataFrame(df.corr())[pd.DataFrame(abs(df.corr())) > thr].notnull()*1).sum().sum() - len(pd.DataFrame(df.corr()).index))/2
def threshold_corr(df, thr = 0.5):
    corr_matrix = pd.DataFrame(df.corr().fillna(0))
    corr_pos = corr_matrix[corr_matrix > thr].notnull()
    corr_neg = corr_matrix[corr_matrix < -thr].notnull()
    columns = indexes = corr_pos.columns
    for row in range(len(indexes)):
        for col in range(len(columns)):
            if(row <= col):
                pass
            elif(corr_pos.iloc[row, col] == True):
                print(indexes[row], " <-> ", columns[col], "(", corr_matrix.iloc[row, col], ")")
            elif(corr_neg.iloc[row, col] == True):
                print(indexes[row], " <-> ", columns[col], "(", corr_matrix.iloc[row, col], ")")
def find_genres(df):
    genres = []
    names = df["artist_name"]
    for name in names:
        try:
            genres.append(search_artist(name)["genres"])
        except:
            genres.append([])
    series_genre = pd.Series(genres, name = "genres")
    df = pd.concat([df, series_genre], axis = 1)
    return df
def perfect_eval(string):
    try:
        return ast.literal_eval(string)
    except:
        return []
def bygenre(df):
    lists = []
    ids = []
    col = np.array([int(x == "genres") for x in df.columns]).argmax()
    for i, ser in enumerate(df["genres"]):
        strlist = perfect_eval(ser)
        if(len(strlist) > 1):
            ids.append(i)
        for genre in strlist:
            df = df.copy()
            df.iloc[i, col] = genre
            lists.append(df.iloc[i])
    dfs = pd.DataFrame(lists)
    df = df.append(dfs)
    df = df.groupby('genres').agg(agg)
    return df
def top_features(df):
    dictall = {}
    for ft in ["acousticness", "danceability", "duration_ms", "energy", "instrumentalness", "liveness", "loudness", "speechiness", "tempo", "valence", "popularity"]:
        dictfts = []
        dfs = df.sort_values(by = ft, ascending = False).iloc[:5][["genres", ft]].set_index("genres")
        for i in range(len(dfs.index)):
            dictft = {}
            dictft.update({dfs.index[i]: dfs.iloc[i][ft]})
            dictfts.append(dictft)
        dictall.update({ft: dictfts})
    return dictall
df = pd.read_csv("spotify_dataset.csv")
bymean = ("acousticness", "danceability", "duration_ms", "energy", "instrumentalness",
           "liveness", "loudness", "speechiness", "tempo", "valence", "popularity")
bymode = ("key", "mode", "time_signature")
bymean_2 = ("acousticness", "danceability", "duration_min", "energy", "instrumentalness",
           "liveness", "loudness", "speechiness", "tempo", "valence", "popularity")
agg = {}
agg_2 = {}
for ft in bymean:
    agg.update({ft: np.mean})
for ft in bymean_2:
    agg_2.update({ft: np.mean})
for ft in bymode:
    agg.update({ft: lambda x: x.value_counts().index[0]})
    agg_2.update({ft: lambda x: x.value_counts().index[0]})
#df_2 = df.groupby("artist_name", as_index = False).agg(agg)
#df_2 = pd.read_csv("spotify_dataset_artist.csv")
#df_2.drop(["Unnamed: 0"], axis = 1, inplace = True) 
#df_4 = find_genres(df_2)
#df_4 = pd.read_csv("spotify_dataset_artist(2).csv")
#df_4.drop("Unnamed: 0", inplace = True, axis = 1)
#df_5 = bygenre(df_4)
#df_5.to_csv("spotify_dataset_genres.csv")

#### Q1: Is popularity dependent on any feature?
#### A1: No significant correlation is found, you can test it by entering the following command:

In [253]:
df.corr()["popularity"]

acousticness       -0.116520
danceability        0.131086
duration_ms        -0.000801
energy              0.122506
instrumentalness   -0.216447
key                 0.002682
liveness           -0.031174
loudness            0.244088
mode               -0.009070
speechiness        -0.000214
tempo               0.037075
time_signature      0.064939
valence             0.014303
popularity          1.000000
Name: popularity, dtype: float64

#### Q2: Can the genres of tracks be grouped based on their similarity?
#### A2: Results from various clustering algorithms indicate that there is no clear distinction between genres. All KMeans, Hierarchical Clustering and KPrototypes algorithms have heterogenous results, that's why it's obvious that genres are tightly interconnected. 

#### Q3: Can the genres be predicted based on audio features?
#### A3: Unfortunately, we cannot make classification for genres based on audio features, because the problems below occurred during this classification procedure: 
#### -Nearly 3000 genres are identified but they're too specific to be general genres, and some of them are either in the minority or least popular, that's why further elimination or regrouping is needed.
#### -The dissimilarity between any two genres happen to be too fuzzy or insignificant, so the probability of classification results being false is high.
#### -Predicted results vary every single time, so the validity of results is questionable.

#### Q4: Which genres are at the top of single audio features?
#### A4: top_features() function returns the top 5 genres for each numeric audio feature, you can test it:

In [256]:
df_g = pd.read_csv("spotify_dataset_genres.csv")
pd.DataFrame(top_features(df_g))

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,{'ukhc': 0.995},{'scratch': 0.9213333333333332},{'reading': 1941467.0},{'birdsong': 0.999},{'chinese traditional': 0.983},{'boogie-woogie': 0.982},{'sarangi': 0.073},{'comedia': 0.8155199999999999},{'8-bit': 200.296},{'neo-rockabilly': 0.995},{'funk bh': 81.0}
1,{'paidika tragoudia': 0.995},{'amharic pop': 0.914},{'comedia': 1673268.58},{'sound effects': 0.997},{'sound effects': 0.978},{'beatdown': 0.952},{'japanese electronic': -1.314},{'fake': 0.531},{'german stoner rock': 199.96400000000003},{'football': 0.978},{'pinoy pop punk': 78.0}
2,{'celtic harp': 0.995},{'kaneka': 0.908},{'hypnosis': 1242834.95},{'vegan straight edge': 0.997},{'paidika tragoudia': 0.97},{'sound effects': 0.8859999999999999},{'neurostep': -1.5347058823529414},{'classic portuguese pop': 0.509},{'shaabi': 195.022},{'medieval ensemble': 0.975},{'belo horizonte indie': 68.0}
3,{'spanish renaissance': 0.994},{'latvian pop': 0.905},{'violin': 898266.0},{'belgian black metal': 0.9965},{'birdsong': 0.966},{'birdsong': 0.86},{'funk bh': -1.624},{'comedy': 0.5045578054298643},{'african electronic': 191.977},{'post-disco soul': 0.973},{'sound team': 68.0}
4,{'danish contemporary classical': 0.994},{'korean electronic': 0.901},{'sound art': 887418.85},{'deep darkpsy': 0.995},{'italian occult psychedelia': 0.963},{'miami bass': 0.7759999999999999},{'kawaii metal': -1.81338835978836},{'clean comedy': 0.5026},{'croatian pop': 181.956},{'bossa nova jazz': 0.966},{'deep rai': 64.0}
